In [1]:
import os
import pandas as pd
import plotly_express as px

In [ ]:
os.getcwd()

In [ ]:
df_sep = pd.read_excel('https://portal.amfiindia.com/spages/amsep2022repo.xls')

In [ ]:
df_sep.columns = ['Sr', 'Category', 'NumSchemes', 'NumFolios', 'InFlow', 'OutFlow', 'NetInFlow', 'NetAUM', 'AverageAUM', 'NumSegPortfolios', 'AUMSegPortfolios']
df_s = df_sep.iloc[2:, 1:]

In [ ]:
df_category = pd.read_csv('../mf_categories.csv')

In [ ]:
df_x = df_s.merge(df_category, on='Category')
df_x['Month'] = 9
df_x['Year'] = 2022

In [ ]:
df_x

In [ ]:
df_filenames = pd.read_excel('../file_names.xlsx')
df_category = pd.read_csv('../mf_categories.csv')

In [ ]:
import datetime
import plotly_express as px
from pandas.tseries.offsets import MonthEnd

In [ ]:
list_monthly = []
for index, row in df_filenames.iterrows():
    if index < 12:
        continue        
    url = 'https://portal.amfiindia.com/spages/' + row['File']
    df_month_data = pd.read_excel(url)
    df_month_data.columns = ['Sr', 'Category', 'NumSchemes', 'NumFolios', 'InFlow', 
                             'OutFlow', 'NetInFlow', 'NetAUM', 'AverageAUM', 
                             'NumSegPortfolios', 'AUMSegPortfolios']
    df_data = df_month_data.iloc[2:, 1:]
    df_data = df_data.merge(df_category, on='Category')
    df_data.loc[df_data['NetAUM'] == df_data[df_data['Category'] == 'ELSS'].groupby(['Class'])['NetAUM'].max().values[0], 'Structure'] = 'Open Ended Schemes'
    df_data.loc[df_data['NetAUM'] == df_data[df_data['Category'] == 'ELSS'].groupby(['Class'])['NetAUM'].min().values[0], 'Structure'] = 'Close Ended Schemes'
    df_data = df_data.drop_duplicates()
    df_data['MonthNumber'] = row['MonthNumber']
    df_data['Year'] = row['Year']
    df_data['MonthName'] = datetime.date(1900, row['MonthNumber'], 1).strftime('%B')
    list_monthly.append(df_data)

In [ ]:
df_mf_aum = pd.concat(list_monthly)

In [ ]:
df_mf_aum['mn'] = df_mf_aum['MonthNumber'].astype(str)
df_mf_aum['mn'] = df_mf_aum['mn'].map(lambda x: x.zfill(2))
df_mf_aum['ymn'] = df_mf_aum['Year'].astype(str) + df_mf_aum['mn']
df_mf_aum['Date'] = pd.to_datetime(df_mf_aum['ymn'], format="%Y%m") + MonthEnd(0)

In [ ]:
df_mf_aum.to_csv('../mf_aum.csv')

In [ ]:
sel_structure = 'Open Ended Schemes'
sel_class = 'Growth/Equity Oriented Schemes'
sel_measure = 'NetInFlow'
df_sel = df_mf_aum[(df_mf_aum['Class'] == sel_class) & (df_mf_aum['Structure'] == sel_structure)][['Category', sel_measure, 'Date']]
df_sel_long = pd.melt(df_sel, id_vars=['Category', 'Date'], value_vars=[sel_measure])
px.line(df_sel_long, x='Date', y='value', color='Category', labels={'value':sel_measure+'(Cr)'})

In [ ]:
df_inflows = pd.DataFrame(df_mf_aum.groupby(['Structure', 'Class', 'Date'])['NetInFlow'].aggregate('sum')).reset_index()

In [ ]:
df_inflows.head()

In [ ]:
sel_structure = 'Open Ended Schemes'
sel_class = 'Growth/Equity Oriented Schemes'
#sel_class = 'Income/Debt Oriented Schemes'
sel_measure = 'NetInFlow'
df_selected = df_inflows[(df_inflows['Class'] == sel_class) & (df_inflows['Structure'] == sel_structure)]
px.line(df_selected, x='Date', y=sel_measure, color='Class', labels={sel_measure:sel_measure + '(Cr)'})